# Testing Pipelines

### Initiating observation via Phoenix

In [4]:
# %pip pip install arize-phoenix
# %pip install llama-index-callbacks-arize-phoenix
# observability
import llama_index.core  # type: ignore
import phoenix as px  # type: ignore

px.launch_app()

llama_index.core.set_global_handler("arize_phoenix", endpoint="http://localhost:6006/v1/traces")

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [1]:
example_inputs = [
    "Die Osterweiterung hat die EU-Institutionen nachhaltig geschwächt.",
    #"In den knapp 70 Jahren seit ihrer Gründung hat es in der Bundeswehr immer wieder rechtsextremistische Vorfälle gegeben.",
    #"In der Bundeswehr gibt es keinen politischen Extremismus.",
]

# example_inputs = [
#     "Current climate models (CMIP6) have difficulties in representing clouds.",
#     "Due to climate change, there is less spring snow cover in Europe.",
#     "Northern Hemisphere spring snow cover is decreasing because of anthropogenic climate change.",
#     "Carbon dioxide removal (CDR) has no substantial side-effects.", # 99
#     "Carbon dioxide removal (CDR) is highly effective and safe.",
#     "Climate change is the greatest threat to humanity.",
#     "An immediate global ban of fossil fuels is the absolutely only way to avoid climate armageddon.",
# ]

In [ ]:
from evidence_seeker import (
    EvidenceSeeker,
    RetrievalConfig,
    ClaimPreprocessingConfig,
    ConfirmationAnalyzerConfig,
)


preprocessing_config = ClaimPreprocessingConfig(
    timeout=1200,
    env_file="../.env",
)
retrieval_config = RetrievalConfig(
    index_persist_path="../TMP/APUZ/storage",
    # Local model (via Huggingface API)
    embed_backend_type="huggingface",
    embed_model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    api_key_name="hf_debatelab_inference_provider",
    env_file="../.env",
)
pipeline = EvidenceSeeker(
    retrieval_config=retrieval_config,
    preprocessing_config=preprocessing_config
)

2025-06-25 07:55:19.182 | INFO     | evidence_seeker.preprocessing.config:load_env_file:56 - Loaded environment variables from '../.env'
2025-06-25 07:55:19.183 | WARNING  | evidence_seeker.retrieval.config:check_hub_token_name:109 - Check whether you need a HF hub token for saving/loading your index to/from the Hugging Face Hub. If you need one, set an `hub_key_name` in the retriever config and provide the token as an environment variable with that name.
2025-06-25 07:55:19.184 | INFO     | evidence_seeker.retrieval.config:load_env_file:155 - Loaded environment variables from '../.env'
2025-06-25 07:55:19.185 | DEBUG    | evidence_seeker.backend:get_openai_llm:236 - Fetching api key via env var: hf_debatelab_inference_provider
2025-06-25 07:55:19.185 | DEBUG    | evidence_seeker.backend:get_openai_llm:249 - Instantiating OpenAILike model (model: meta-llama/Llama-3.2-3B-Instruct-Turbo,base_url: https://router.huggingface.co/together/v1).
No sentence-transformers model found with name s

OSError: There was a specific connection error when trying to load sentence-transformers/paraphrase-multilingual-mpnet-base-v2:
401 Client Error: Unauthorized for url: https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2/resolve/main/config.json (Request ID: Root=1-685b8f47-3d76e3fb7cc45c1a512d87dd;aca83c7b-cff3-4713-b161-d9500cadf9f6)

Invalid credentials in Authorization header

In [3]:
from IPython.display import display, Markdown  # type: ignore

def describe_results(claim: str, results: list):
    preamble_template = (
        '## EvidenceSeeker Results\n\n'
        '### Input\n\n'
        '**Submitted claim:** {claim}\n\n'
        '### Results\n\n'
    )
    result_template = (
        '**Clarified claim:** <font color="orange">{text}</font> [_{statement_type}_]\n\n'
        '**Status**: {verbalized_confirmation}\n\n'
        '|Metric|Value|\n'
        '|:---|---:|\n'
        '|Average confirmation|{average_confirmation:.3f}|\n'
        '|Evidential divergence|{evidential_uncertainty:.3f}|\n'
        '|Width of evidential base|{n_evidence}|\n\n'
    )
    markdown = []
    markdown.append(preamble_template.format(claim=claim))
    for claim_dict in results:
        rdict = claim_dict.copy()
        rdict["statement_type"] = rdict["statement_type"].value
        markdown.append(result_template.format(**claim_dict))
    display(Markdown("\n".join(markdown)))


### 02.06.2025


In [4]:
for example_input in example_inputs:
    results = await pipeline(example_input)
    describe_results(example_input, results)
    display(Markdown("------\n"))

2025-06-02 10:34:38.179 | DEBUG    | evidence_seeker.preprocessing.workflows:ascriptive_analysis:173 - Analysing ascriptive aspects of claim.
2025-06-02 10:34:38.489 | DEBUG    | evidence_seeker.preprocessing.workflows:descriptive_analysis:118 - Analysing descriptive aspects of claim.
2025-06-02 10:34:38.512 | DEBUG    | evidence_seeker.preprocessing.workflows:normative_analysis:228 - Analysing normative aspects of claim.
2025-06-02 10:34:43.601 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:281 - Negating claim.
2025-06-02 10:34:43.607 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:281 - Negating claim.
2025-06-02 10:34:44.161 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:281 - Negating claim.
2025-06-02 10:34:44.168 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:281 - Negating claim.
2025-06-02 10:34:44.187 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:281 - Negating claim.
2025-06-02 10:

## EvidenceSeeker Results

### Input

**Submitted claim:** Die Osterweiterung hat die EU-Institutionen nachhaltig geschwächt.

### Results


**Clarified claim:** <font color="orange">Die EU-Kommission hat die Osterweiterung nachhaltig geschwächt.</font> [_StatementType.ASCRIPTIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">Die EU-Institutionen haben die Osterweiterung nachhaltig geschwächt.</font> [_StatementType.ASCRIPTIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">Die EU-Institutionen sind nachhaltig geschwächt worden</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">Die Osterweiterung hat die EU-Institutionen nachhaltig geschwächt</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">Die Osterweiterung ist eine negative Entwicklung für die EU-Institutionen gewesen</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">Die EU-Institutionen wurden nachhaltig geschwächt.</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">Die Osterweiterung führte zu einer Veränderung der Machtverteilung innerhalb der EU-Institutionen.</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">Die Osterweiterung war ein historisches Ereignis, das im 2000er Jahren stattfand.</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|



------


### 12/12/2024

In [ ]:
for example_input in example_inputs:
    results = await pipeline(example_input)
    describe_results(example_input, results)
    display(Markdown("------\n"))

2025-05-15 16:00:51.825 | DEBUG    | evidence_seeker.preprocessing.workflows:ascriptive_analysis:174 - Analysing ascriptive aspects of claim.
2025-05-15 16:00:51.837 | DEBUG    | evidence_seeker.preprocessing.workflows:descriptive_analysis:118 - Analysing descriptive aspects of claim.
2025-05-15 16:00:51.841 | DEBUG    | evidence_seeker.preprocessing.workflows:normative_analysis:229 - Analysing normative aspects of claim.
2025-05-15 16:01:33.818 | DEBUG    | evidence_seeker.preprocessing.workflows:list_ascriptive_claims:212 - Number of ascriptive claims: 1
2025-05-15 16:01:33.820 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:282 - Negating claim.
2025-05-15 16:01:38.399 | DEBUG    | evidence_seeker.preprocessing.workflows:list_descriptive_claims:156 - Number of descriptive claims: 2
2025-05-15 16:01:38.400 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:282 - Negating claim.
2025-05-15 16:01:38.404 | DEBUG    | evidence_seeker.preprocessing.workflo

ValueError: Confirmation analysis failed.

2025-05-15 16:25:18.358 | DEBUG    | evidence_seeker.confirmation_analysis.workflows:collect_analyses:230 - Probs for the claim: [0.0, 0.0, 0.0]
2025-05-15 16:25:18.358 | DEBUG    | evidence_seeker.confirmation_analysis.workflows:collect_analyses:231 - Probs for the negation: [1.0, 0.0, 0.0]
2025-05-15 16:25:18.360 | DEBUG    | evidence_seeker.confirmation_analysis.workflows:freetext_analysis:117 - Confirmation analysis.
2025-05-15 16:25:18.363 | DEBUG    | evidence_seeker.confirmation_analysis.workflows:freetext_analysis:117 - Confirmation analysis.
2025-05-15 16:25:25.784 | DEBUG    | evidence_seeker.confirmation_analysis.workflows:collect_analyses:230 - Probs for the claim: []
2025-05-15 16:25:25.785 | DEBUG    | evidence_seeker.confirmation_analysis.workflows:collect_analyses:231 - Probs for the negation: [0.0, 0.0, 1.0]
2025-05-15 16:25:25.785 | ERROR    | evidence_seeker.confirmation_analysis.workflows:collect_analyses:234 - Confirmation analysis failed.
2025-05-15 16:25:44.977 |